In [2]:
from pathlib import Path
import yaml
import pandas as pd

config_path = Path("../config.yaml")
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

csv_relative_path = config["data"]["clean_data_csv"]["clean_data_v3_irma"]
csv_path = config_path.parent / csv_relative_path

df = pd.read_csv(csv_path)
print(df.shape)
df.head()



(7566, 46)


,selling_price,km_driven,owner,mileage,engine,max_power,torque,seats,fuel_Diesel,fuel_LPG,...,brand_Nissan,brand_Opel,brand_Peugeot,brand_Renault,brand_Skoda,brand_Tata,brand_Toyota,brand_Volkswagen,brand_Volvo,car_age
0,0.488000,0.341286,0.0,0.655999,0.535379,0.467401,0.418743,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
1,0.435603,0.316742,1.0,0.582041,0.640637,0.627256,0.498515,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,11
2,0.206186,0.336228,2.0,0.472299,0.640276,0.493676,0.040177,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19
3,0.302060,0.323773,0.0,0.642808,0.601599,0.562794,0.081435,5.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15
4,0.152797,0.316742,0.0,0.422550,0.559201,0.553238,0.034175,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18


In [3]:
# Define target and features
X = df.drop(columns=["selling_price"])
y = df["selling_price"]


In [4]:
# Train/Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)


(6052, 45) (1514, 45)


In [5]:
#Feature Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model(name, model,
                   X_train_scaled, X_test_scaled,
                   X_train, X_test,
                   y_train, y_test, use_scaled=True):
    """
    Train and evaluate a regression model and store results in the global results_df.
    Automatically removes any previous entry of the same model name to avoid duplicates.
    """

    # Select scaled or unscaled data depending on the model
    if use_scaled:
        Xtr, Xte = X_train_scaled, X_test_scaled
    else:
        Xtr, Xte = X_train, X_test

    # Train the model
    model.fit(Xtr, y_train)
    y_pred = model.predict(Xte)

    # Compute metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # Update global results without duplicates
    global results_df
    results_df = results_df[results_df["Model"] != name]  # remove old row if it exists

    new_row = pd.DataFrame([{
        "Model": name,
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R²": r2
    }])

    results_df = pd.concat([results_df, new_row], ignore_index=True)

    # Display summary in console
    print(f"{name} -> R²: {r2:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f} | MSE: {mse:.4f}")


In [19]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Create model KNN before scaling
knn_raw = KNeighborsRegressor()

# Train and predict 
knn_raw.fit(X_train, y_train)
y_pred_knn_raw = knn_raw.predict(X_test)

# Compute metrics
mae_raw = mean_absolute_error(y_test, y_pred_knn_raw)
mse_raw = mean_squared_error(y_test, y_pred_knn_raw)
rmse_raw = np.sqrt(mse_raw)
r2_raw = r2_score(y_test, y_pred_knn_raw)

print("KNN (sin escalar)")
print(f"MAE: {mae_raw:.2f}")
print(f"MSE: {mse_raw:.2f}")
print(f"RMSE: {rmse_raw:.2f}")
print(f"R²: {r2_raw:.4f}")


KNN (sin escalar)
MAE: 0.05
MSE: 0.00
RMSE: 0.06
R²: 0.8886


In [15]:
# Create model KNN after scaling
knn_scaled = KNeighborsRegressor()

# train with scaled data
knn_scaled.fit(X_train_scaled, y_train)
y_pred_knn_scaled = knn_scaled.predict(X_test_scaled)

# compute metrics
mae_scaled = mean_absolute_error(y_test, y_pred_knn_scaled)
mse_scaled = mean_squared_error(y_test, y_pred_knn_scaled)
rmse_scaled = np.sqrt(mse_scaled)
r2_scaled = r2_score(y_test, y_pred_knn_scaled)

print("KNN (con datos escalados)")
print(f"MAE: {mae_scaled:.2f}")
print(f"MSE: {mse_scaled:.2f}")
print(f"RMSE: {rmse_scaled:.2f}")
print(f"R²: {r2_scaled:.4f}")


KNN (con datos escalados)
MAE: 0.04
MSE: 0.00
RMSE: 0.06
R²: 0.8894


### KNN Before vs After Scaling

We tested the KNN model both before and after feature scaling.  
Because the dataset had already been normalized during the cleaning phase (values between 0 and 1),  
the results were almost identical (R² ≈ 0.89).  
This confirms that the features were already scaled properly and no additional normalization was required.


In [17]:
# Linear Regression (after scaling)
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

evaluate_model("Linear Regression", lin_reg, 
               X_train_scaled, X_test_scaled, 
               X_train, X_test, 
               y_train, y_test, use_scaled=True)


NameError: name 'results_df' is not defined

In [20]:
#Bagging Regressor (before scaling)

from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

bagging = BaggingRegressor(
    estimator=DecisionTreeRegressor(random_state=42),  # ← cambio aquí
    n_estimators=100,
    bootstrap=True,   # True = Bagging (con reemplazo)
    random_state=42
)

evaluate_model("Bagging (DecisionTree)", bagging,
               X_train_scaled, X_test_scaled, 
               X_train, X_test,
               y_train, y_test, use_scaled=False)


NameError: name 'results_df' is not defined

In [22]:
#Pasting Regressor (before scaling)
pasting = BaggingRegressor(
    estimator=DecisionTreeRegressor(random_state=42),  # ← cambio aquí también
    n_estimators=100,
    bootstrap=False,   # False = Pasting (sin reemplazo)
    random_state=42
)

evaluate_model("Pasting (DecisionTree)", pasting,
               X_train_scaled, X_test_scaled, 
               X_train, X_test,
               y_train, y_test, use_scaled=False)



NameError: name 'results_df' is not defined

In [ ]:
# Random Forest (before scaling)
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=42
)

evaluate_model("Random Forest", rf,
               X_train_scaled, X_test_scaled, 
               X_train, X_test,
               y_train, y_test, use_scaled=False)


In [ ]:
# Gradient Boosting (before scaling)
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.1,
    random_state=42
)

evaluate_model("Gradient Boosting", gbr,
               X_train_scaled, X_test_scaled, 
               X_train, X_test,
               y_train, y_test, use_scaled=False)


In [23]:
#Adaptive Boosting (AdaBoost) before scaling
from sklearn.ensemble import AdaBoostRegressor

ada = AdaBoostRegressor(
    n_estimators=200,
    learning_rate=0.8,
    random_state=42
)

evaluate_model("AdaBoost", ada,
               X_train_scaled, X_test_scaled, 
               X_train, X_test,
               y_train, y_test, use_scaled=False)


NameError: name 'results_df' is not defined

In [24]:
results_df.sort_values(by="R²", ascending=False).reset_index(drop=True)


NameError: name 'results_df' is not defined

In [ ]:
import matplotlib.pyplot as plt

tmp = results_df.sort_values(by="R²", ascending=True)

plt.figure(figsize=(8,5))
plt.barh(tmp["Model"], tmp["R²"])
plt.title("Model Performance (R²)")
plt.xlabel("R²")
plt.show()

plt.figure(figsize=(8,5))
plt.barh(tmp["Model"], tmp["RMSE"])
plt.title("Model Error (RMSE)")
plt.xlabel("RMSE")
plt.show()


### Model Evaluation Summary

We evaluated several regression models to predict car selling prices.  
The following metrics were used: **MAE**, **MSE**, **RMSE**, and **R²**.

**Key findings:**
- **Random Forest** achieved the best overall performance (R² = 0.93), indicating strong predictive power and stability.  
- **Bagging (Decision Tree)** performed nearly as well, confirming the benefit of ensemble averaging.  
- **Gradient Boosting** also showed robust performance, slightly below Random Forest.  
- **Linear Regression** and **KNN** achieved decent but weaker results, as expected for non-ensemble methods.  
- **AdaBoost** underperformed, possibly due to sensitivity to outliers and non-linear relationships.

In conclusion, **Random Forest** was selected as the best model for further optimization (hyperparameter tuning) because it provides the best balance between bias and variance.
